In [20]:
import folium
import openrouteservice
from geopy.geocoders import Nominatim
import speech_recognition as sr
import pyttsx3

engine = pyttsx3.init()

In [26]:
import pandas as pd

file_path = "project cs.xlsx"  
df = pd.read_excel(file_path)
df.head()

,room,name,direction,department
0,SJ101,Library,"Go to ground floor of St Joseph block, library...",cs
1,SJ110,Seminar Hall,"From the cafeteria, go to back entrance of St ...",cs
2,SJ201,S3/S4 R2,"On the first floor, turn left, class will be t...",cs
3,SJ202,Faculty room,"On the first floor, turn left, faculty room wi...",cs
4,SJ203,S7/S8 R2,"turn right, first class on the left side of ha...",cs


In [28]:
def check_room(df,string):
    flag=0
    direction=''
    departm=''
    for _, row in df.iterrows():
        
        name = row["name"].lower()
        direction = row["direction"]
        departm = row["department"]
        if name in string.lower():
            return True,direction,departm
    return flag,direction,departm

In [30]:
def recognize_speech():
    recognizer = sr.Recognizer()
    
    with sr.Microphone() as source:
        print("Adjusting for ambient noise, please wait...")
        recognizer.adjust_for_ambient_noise(source, duration=1)
        print("Listening for your speech...")
        
        try:
            audio = recognizer.listen(source, timeout=5)
            print("Recognizing...")
            
            # Recognize speech using Google's Speech Recognition API
            text = recognizer.recognize_google(audio)
            print(f"You said: {text}")
            return text
        
        except sr.WaitTimeoutError:
            print("Listening timed out. Please try again.")
        except sr.UnknownValueError:
            print("Could not understand the audio. Please try again.")
        except sr.RequestError as e:
            print(f"Could not request results from the service; {e}")

def find_coordinates(input_string, places_dict):
    for place, coordinates in places_dict.items():
        if place in input_string:
            return coordinates
    return None 
def geocode_location(location_name):
    geolocator = Nominatim(user_agent="geoapi")
    location = geolocator.geocode(location_name)
    if location:
        return (location.longitude, location.latitude)
    else:
        raise ValueError(f"Location '{location_name}' could not be geocoded.")

def create_walking_route_map(api_key, coordinates):

    start_coords=(76.85983445667758, 8.558296623824283)
    end_coords = coordinates

    client = openrouteservice.Client(key=api_key)

    route = client.directions(
        coordinates=[start_coords, end_coords],
        profile='foot-walking',
        format='geojson'
    )

    route_coords = route['features'][0]['geometry']['coordinates']

    route_coords = [(coord[1], coord[0]) for coord in route_coords]

    midpoint = (
        (start_coords[1] + end_coords[1]) / 2,
        (start_coords[0] + end_coords[0]) / 2
    )
    start_location_name="You are here"
    end_location_name="Destination"
    route_map = folium.Map(location=midpoint, zoom_start=13)

    folium.Marker(location=(start_coords[1], start_coords[0]), popup=start_location_name, icon=folium.Icon(color="green")).add_to(route_map)
    folium.Marker(location=(end_coords[1], end_coords[0]), popup=end_location_name, icon=folium.Icon(color="red")).add_to(route_map)

    folium.PolyLine(locations=route_coords, color="blue", weight=5).add_to(route_map)

    return route_map

if __name__ == "__main__":
    API_KEY = "5b3ce3597851110001cf6248cc71b5f3b13b488a8a823f8bb89a91af"
    coordinates=0
    places = {
        'mechanical':(76.86000938250643, 8.558715693710296),
        'canteen':(76.86062860490402, 8.559007450953514),
        'electronics':(76.85977006628079, 8.559102935093623),
        'basketball':(76.86040592144863, 8.557649451701682),
        'civil':(76.86086738668281, 8.55798629962408),
        'auditorium':(76.86054516385475, 8.559458348071377),
        'boys':(76.86136077554683, 8.559384082700474),
        'girls':(76.85913394099283, 8.557771459645002),
        'electrical':(76.85913394099283, 8.557771459645002),
        'cs':( 76.860546,8.558302)
    }
    
    input_string = recognize_speech()
    flag,direction,departm=check_room(df,input_string)
    
    if flag:
        coordinates = find_coordinates(departm, places)
        engine.say(direction)
        engine.runAndWait()
    else:
        coordinates = find_coordinates(input_string.lower(), places)
    
    if coordinates:
        print(f"Coordinates for the place found: {coordinates}")
        try:
            route_map = create_walking_route_map(API_KEY, coordinates)
    
            route_map.save("walking_route_map.html")
            print("Walking route map saved as walking_route_map.html")
        except ValueError as e:
            print(e)
    else:
        print("No place found in the input string.")


Adjusting for ambient noise, please wait...
Listening for your speech...
Recognizing...
You said: how do you reach the Libra
No place found in the input string.
